### 8.6 행렬 연산을 통한 기학 변환 - 어파인 변환

어파인 변환 : 2x3 행렬로 변환 행렬을 구성

-> 계산이 불가능하므로 3x3 행렬로 만든 후, 마지막 행을 삭제

In [2]:
# 8.6.1 어파인 변환
import numpy as np, cv2
from Common.functions import contain
from Common.interpolation import bilinear_value

# 어파인 변환 실행 함수
def affine_transform(img, mat):
    rows, cols = img.shape[:2]
    inv_mat = cv2.invertAffineTransform(mat)

    pts = [np.dot(inv_mat, (j, i, 1)) for i in range(rows) for j in range(cols)]
    dst = [bilinear_value(img, p) if contain(p, size) else 0 for p in pts]
    dst = np.reshape(dst, (rows, cols)).astype('uint8')

    return dst

image = cv2.imread("images_08/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

center = (200, 200)
angle, scale = 30, 1
size = image.shape[::-1]

pt1 = np.array([(30, 70), (20, 240), (300, 110)], np.float32)
pt2 = np.array([(120, 20), (10, 180), (280, 260)], np.float32)
aff_mat = cv2.getAffineTransform(pt1, pt2)
rot_mat = cv2.getRotationMatrix2D(center, angle, scale)

dst3 = cv2.warpAffine(image, aff_mat, size, cv2.INTER_LINEAR)
dst4 = cv2.warpAffine(image, rot_mat, size, cv2.INTER_LINEAR)

image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
dst3 = cv2.cvtColor(dst3, cv2.COLOR_GRAY2BGR)

for i in range(len(pt1)):
    cv2.circle(image, tuple(pt1[i].astype(int)), 3, (0, 0, 255), 2)
    cv2.circle(dst3, tuple(pt2[i].astype(int)), 3, (0, 0, 255), 2)

cv2.imshow("image", image)
cv2.imshow("cv affine", dst3)
cv2.imshow("cv affine rotate", dst4)
cv2.waitKey(0)


    



-1

In [1]:
# 8.6.2 어파인 변환 연결
import numpy as np, cv2
from Common.interpolation import affine_transform

def getAffineMat(center, degree, fx=1, fy=1, translate=(0,0)):
    scale_mat = np.eye(3, dtype=np.float32)
    cen_trans = np.eye(3, dtype=np.float32)
    org_trans = np.eye(3, dtype=np.float32)
    trans_mat = np.eye(3, dtype=np.float32)
    rot_mat = np.eye(3, dtype=np.float32)

    radian = degree / 180 * np.pi
    rot_mat[0] = [ np.cos(radian), np.sin(radian), 0 ]
    rot_mat[1] = [ -np.sin(radian), np.cos(radian), 0 ]

    cen_trans[:2, 2] = center
    org_trans[:2, 2] = -center[0], -center[1]
    trans_mat[:2, 2] = translate
    scale_mat[0, 0], scale_mat[1, 1] = fx, fy

    ret_mat = cen_trans.dot(rot_mat.dot(trans_mat.dot(scale_mat.dot(org_trans))))
    return np.delete(ret_mat, 2, axis=0)

image = cv2.imread("images_08/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

size = image.shape[::-1]
center = np.divmod(size, 2)[0]
angle, tr = 45, (200, 0)

aff_mat1 = getAffineMat(center, angle)
aff_mat2 = getAffineMat((0, 0), 0, 2.0, 1.5)
aff_mat3 = getAffineMat(center, angle, 0.7, 0.7)
aff_mat4 = getAffineMat(center, angle, 0.7, 0.7, tr)

dst1 = cv2.warpAffine(image, aff_mat1, size, cv2.INTER_LINEAR)
dst2 = cv2.warpAffine(image, aff_mat2, size, cv2.INTER_LINEAR)
dst3 = affine_transform(image, aff_mat3, size)
dst4 = affine_transform(image, aff_mat4, size)

cv2.imshow("image", image)
cv2.imshow("rotate", dst1)
cv2.imshow("scaling", dst2)
cv2.imshow("rotate_scaling", dst3)
cv2.imshow("rotate_scaling_tanslate", dst4)
cv2.waitKey(0)


    



-1

In [ ]:
# 8.6.3 마우스 드래그로 영상 회전하기
import numpy as np, cv2

def contain_pts(p, p1, p2):
    return p1[0] <= p[0] < p2[0] and p1[1] <= p[1] < p2[1]

def draw_rect(title, img, pts):
    rois = [(p - small, small * 2) for p in pts]
    for (x, y), (w, h) in np.int32(rois):
        cv2.rectangle(img, (x, y, w, h), (0, 255, 0), 2)
    cv2.imshow(title, img)

def affine(img):